In [82]:
import numpy as np
import pandas as pd
import random
import math
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

In [103]:
def read_data(exp_num):
    print(f"Data Set: Experiment{exp_num}")
    data_frame = pd.read_csv(f"experiments/Experiment{exp_num}.txt")
    nom_assignments = data_frame['m']
    new_df = data_frame.drop(['m'],axis=1)
    gaus_data = new_df.to_numpy()
    N,d = gaus_data.shape
    return gaus_data, N, d, nom_assignments.to_numpy()

In [80]:
def distance_between_points(a, b):
    try:
        dim = len(b)
        dim = len(a)
    except TypeError:
        return (a-b) * (a-b)
    s = 0
    for i in range(0, dim):
        s+=(a[i] - b[i]) * (a[i] - b[i])
    return s

In [89]:
def likelihood_of_pairs(mean_assignments, nominal_assignments):
    #group nominal assignments
    likelihoods = []
    for cluster, data_idx in enumerate(mean_assignments):
        same_source = 0
        total_in_cluster = 0
        #Split data_idx into two sublists and stack them
        try:
            sublists = np.split(np.array(data_idx), 2)
        except ValueError:
            data_idx.pop()
            sublists = np.split(np.array(data_idx),2)
         #stacking!
        stacked_list = np.stack(sublists, axis=1)
        #now compare the two nominal assignments
        for pair in stacked_list:
            #print(f"pair {pair}, {assign[pair[0]]} == {assign[pair[1]]}")
            if nominal_assignments[pair[0]] == nominal_assignments[pair[1]]:
                same_source += 1
            total_in_cluster += 1
        #print(f"Cluster {cluster} | same sources {same_source} / {total_in_cluster}")
        if total_in_cluster == 0:
            likelihoods.append(0)
        else:
            likelihoods.append(same_source/total_in_cluster)
    return np.mean(likelihoods)
    

In [12]:
class GMM:
    def __init__(self, data, k, N, dim, n_iterations = 100):
        self.data = data
        self.k = k
        self.N = N
        self.dimension = dim
        self.n_iterations = n_iterations
        self.run_weights = []
        self.weight_total = []
    
    def calculate_posterior(self):
        distributions = []
        for i in range(0, self.k):
            distributions.append(multivariate_normal(mean = self.mu[i], cov = self.sigma[i]).pdf(self.data))
        gaus_densities = np.array(distributions).T
        num = gaus_densities * self.phi
        den = num.sum(axis=1)[:, np.newaxis]
        return num/den
    
    def update_mean(self, weight, weight_sum):
        #FOR EACH WEIGHT, multiply by the weight and take summation divided by the total weight
        #h = (self.data * weight).sum(axis=0)/weight_total
        weight_data = self.data * weight 
        return np.sum(weight_data, axis=0) / weight_sum
    
    def update_cov(self, weight, weight_sum):
        #multiply weight by (data - mean)(data-mean)transposed divide by total weight
        #NP COV MAKES A GREAT FUNCTION FOR THIS!
        #A WEIGHTS ARE THE VECTOR WEIGHTS USED TO ASSIGN PROBABILITIES TO OBSERVATIONS!
        #BIAS CORRECTS THE NORMALIZATION FEATURE
        cov_matrix_weights = weight / weight_sum
        x2 = np.cov(self.data.T, aweights = cov_matrix_weights.flatten(), bias=True)
        return x2

    def avg_means_over_runs(self, n_runs = 50):
        for i in range(0, n_runs):
            self.calculate_means()
            self.mu.sort()
            self.run_weights.append(self.mu)
        return np.mean(self.run_weights, axis=0)
    
    def calculate_means(self):
        #initialize phi and the covariance matrix, randomly initialize means, 
        self.phi = np.array([(1/self.k) for i in range(0,self.k)])
        self.mu = random.sample(sorted(self.data), self.k)
        self.sigma = [np.cov(self.data.T) for i in range(0, self.k)]
        
        for i in range(0, self.n_iterations):
            self.weights = self.calculate_posterior()
            #print(f"WEIGHTS {self.weights.shape}")
            self.phi = np.mean(self.weights, axis = 0)
            for j in range(0,self.k):
                weight_j = self.weights[:,[j]]
                sum_of_weight = np.sum(weight_j)
                self.mu[j] = self.update_mean(weight_j, sum_of_weight)
                self.sigma[j] = self.update_cov(weight_j, sum_of_weight)
                        
    

In [78]:
def get_assignments(means, data, k, distance_calculation):
    assignments = [[] for i in range(0,k)]
    for idx, data_point in enumerate(data):
        distances = [distance_calculation(data_point, m) for m in means]
        optimal_cluster_id = distances.index(min(distances))
        #print(f" data point {data_point} -> optimal {optimal_cluster_id} = {means[optimal_cluster_id]}")
        assignments[optimal_cluster_id].append(idx)
    return assignments

In [10]:
faux_data, N, d, a = read_data(2)
test_gmm = GMM(faux_data, 3, N, d)
test_gmm.avg_means_over_runs(n_runs=50)

array([[1.13111206],
       [1.93597506],
       [2.85140955]])

In [73]:
def calculate_std_dev_euclidean(means, data, k, N, dimension):
    assignments = get_assignments(means, data, k, distance_between_points)
    std_dev = []
    for idx, cluster in enumerate(assignments):
        s = 0
        for idx_c, data_point in enumerate(cluster):
            if dimension == 1:
                s += math.sqrt((means[idx] - data_point) * (means[idx] - data_point))
            else:
                x = 0
                for i in range(0, dimension):
                    x += (means[idx][i] - data_point[i])*(means[idx][i] - data_point[i])
                s+= math.sqrt(x)
        std_dev.append(math.sqrt(s / N))
    return std_dev

In [91]:
#Experiment_1_part_1
standard_dev = 1
S = 3
spacing = [0.5, 1, 1.5, 2]
k_varied = [2, 3, 6, 8]
r_exp1 = []
std_dev_exp1 = []
avg_std_dev_exp1 = []
lop_exp_1 = []
for idx, space in enumerate(spacing):
    for k in k_varied:
        data, N, d, a = read_data(idx+1)
        clf = GMM(data, k, N, d)
        results = clf.avg_means_over_runs()
        print(f"S {S} | spacing {space} | k {k}")
        print(f"Nominal means {np.unique(a)}")
        print(f"Results for {k} | {results}")
        r_exp1.append(results)
        s_d = calculate_std_dev_euclidean(results, data, k, N,d)
        std_dev_exp1.append(s_d)
        mean_assignments = get_assignments(results, data, k, simple_distance)
        #print(mean_assignments)
        lop = likelihood_of_pairs(mean_assignments, a)
        print(f"Standard Deviations {s_d}")
        print(f"Mean Standard Deviation across {k} clusters: {np.mean(s_d)}")
        print(f"Likelihood of pairs {lop}")
        lop_exp_1.append(lop)
        
        print()
        avg_std_dev_exp1.append(np.mean(s_d))

Data Set: Experiment1
S 3 | spacing 0.5 | k 2
Nominal means [0.5 1.  1.5]
Results for 2 | [[0.51399927]
 [1.34777996]]
Standard Deviations [16.15282824684679, 15.41645156765033]
Mean Standard Deviation across 2 clusters: 15.784639907248561
Likelihood of pairs 0.3617847138855542

Data Set: Experiment1
S 3 | spacing 0.5 | k 3
Nominal means [0.5 1.  1.5]
Results for 3 | [[0.24784608]
 [1.00481029]
 [1.56320183]]
Standard Deviations [14.178323043930078, 10.777051578280236, 13.469282005187067]
Mean Standard Deviation across 3 clusters: 12.808218875799128
Likelihood of pairs 0.3521374670086614

Data Set: Experiment1
S 3 | spacing 0.5 | k 6
Nominal means [0.5 1.  1.5]
Results for 6 | [[-0.0801099 ]
 [ 0.45861245]
 [ 0.79993799]
 [ 1.10102125]
 [ 1.37062543]
 [ 1.97659532]]
Standard Deviations [11.643227579317442, 8.100897847227715, 7.874135345617686, 7.151676976617444, 7.698980529634323, 11.179028908539212]
Mean Standard Deviation across 6 clusters: 8.941324531158969
Likelihood of pairs 0.335

In [84]:
def simple_distance(a,b):
    return (a-b)*(a-b)

In [99]:
#Experiment_1_Part_2
exp_num = 4
generators = [5, 10]
for S in generators:
    for space in spacing:
        exp_num+=1
        data, N, d, a = read_data(exp_num)
        clf = GMM(data, S, N, d)
        results = clf.avg_means_over_runs()
        print(f"S {S} | spacing {space}")
        print(f"Nominal means {np.unique(a)}")
        print(f"Results for {S} | {results}")
        #r_exp1.append(results)
        s_d = calculate_std_dev_euclidean(results, data, S, N,d)
        #std_dev_exp1.append(s_d)
        mean_assignments = get_assignments(results, data, S, simple_distance)
        #print(mean_assignments)
        lop = likelihood_of_pairs(mean_assignments, a)
        print(f"Standard Deviations {s_d}")
        print(f"Mean Standard Deviation across {S} clusters: {np.mean(s_d)}")
        print(f"Likelihood of pairs {lop}")
        #lop_exp_1.append(lop)
        print()
        #avg_std_dev_exp1.append(np.mean(s_d))

Data Set: Experiment5
S 5 | spacing 0.5
Nominal means [0.5 1.  1.5 2.  2.5]
Results for 5 | [[0.29593371]
 [1.10327514]
 [1.48116521]
 [2.00738497]
 [2.64846231]]
Standard Deviations [11.220593595672932, 9.20146891126965, 8.65180833978739, 9.08937496038308, 11.400113486215272]
Mean Standard Deviation across 5 clusters: 9.912671858665664
Likelihood of pairs 0.2692805079701791

Data Set: Experiment6
S 5 | spacing 1
Nominal means [1. 2. 3. 4. 5.]
Results for 5 | [[1.13506564]
 [2.21359793]
 [3.21995381]
 [3.79251902]
 [4.46535987]]
Standard Deviations [10.875587094071161, 9.261709927731285, 9.257948360892572, 7.764061619512445, 12.103472461950647]
Mean Standard Deviation across 5 clusters: 9.85255589283162
Likelihood of pairs 0.36781003311452637

Data Set: Experiment7
S 5 | spacing 1.5
Nominal means [1.5 3.  4.5 6.  7.5]
Results for 5 | [[1.32091365]
 [3.41569929]
 [4.43383467]
 [5.3888025 ]
 [7.09299827]]
Standard Deviations [10.440178373886376, 9.884808771144554, 8.133302004299875, 9.38

In [100]:
#Experiment 2
std_dev = [1,2,3]
k=5
for idx, s_dev in enumerate(std_dev):
    data, N, d, a = read_data(f"_2_{idx+1}")
    clf = GMM(data, 5, N, d)
    results = clf.avg_means_over_runs()
    print(f" S {5} | std_dev {s_dev}")
    print(f"Nominal means {np.unique(a)}")
    print(f"Results for {k} | {results}")
    #r_exp1.append(results)
    s_d = calculate_std_dev_euclidean(results, data, k, N,d)
    #std_dev_exp1.append(s_d)
    mean_assignments = get_assignments(results, data, k, simple_distance)
    #print(mean_assignments)
    lop = likelihood_of_pairs(mean_assignments, a)
    print(f"Standard Deviations {s_d}")
    print(f"Mean Standard Deviation across {k} clusters: {np.mean(s_d)}")
    print(f"Likelihood of pairs {lop}")
    #lop_exp_1.append(lop)
    print()
    #avg_std_dev_exp1.append(np.mean(s_d))

Data Set: Experiment_2_1
 S 5 | std_dev 1
Nominal means [ 1.  4.  7. 10. 13.]
Results for 5 | [[ 1.23013747]
 [ 4.695164  ]
 [ 7.36690454]
 [ 9.7786939 ]
 [13.1317965 ]]
Standard Deviations [10.644488696925826, 10.0752844886738, 9.355521990043218, 9.82159375215016, 9.677831961726827]
Mean Standard Deviation across 5 clusters: 9.914944177903967
Likelihood of pairs 0.7912872166065443

Data Set: Experiment_2_2
 S 5 | std_dev 2
Nominal means [ 1.  4.  7. 10. 13.]
Results for 5 | [[ 1.27031469]
 [ 5.11717866]
 [ 7.24940326]
 [ 9.44078634]
 [12.09474481]]
Standard Deviations [11.260557313633784, 9.56058241188646, 7.361254433508194, 9.894055594110354, 11.058404667817326]
Mean Standard Deviation across 5 clusters: 9.826970884191223
Likelihood of pairs 0.47492882605449793

Data Set: Experiment_2_3
 S 5 | std_dev 3
Nominal means [ 1.  4.  7. 10. 13.]
Results for 5 | [[ 2.25451907]
 [ 4.78344167]
 [ 7.03576942]
 [ 8.77827951]
 [12.63827515]]
Standard Deviations [11.657037975868755, 9.121290954810

In [101]:
#Experiment 3
k=5
data, N, d, a = read_data(f"_3_1")
clf = GMM(data, 5, N, d)
results = clf.avg_means_over_runs()
print(f"experiment 3")
print(f"Nominal means {np.unique(a)}")
print(f"Results for {k} | {results}")
#r_exp1.append(results)
s_d = calculate_std_dev_euclidean(results, data, k, N,d)
#std_dev_exp1.append(s_d)
mean_assignments = get_assignments(results, data, k, simple_distance)
#print(mean_assignments)
lop = likelihood_of_pairs(mean_assignments, a)
print(f"Standard Deviations {s_d}")
print(f"Mean Standard Deviation across {k} clusters: {np.mean(s_d)}")
print(f"Likelihood of pairs {lop}")
#lop_exp_1.append(lop)
print()
#avg_std_dev_exp1.append(np.mean(s_d))

Data Set: Experiment_3_1
experiment 3
Nominal means [1.   2.25 3.5  4.75 6.  ]
Results for 5 | [[1.41539216]
 [2.42762337]
 [3.52037572]
 [4.49549109]
 [6.16347022]]
Standard Deviations [11.767763572940378, 7.602093787129054, 8.05708551668375, 9.757348393636951, 11.806715720292631]
Mean Standard Deviation across 5 clusters: 9.798201398136552
Likelihood of pairs 0.3463370108524681



In [ ]:
#Experiment 4
k = 5
N = [100, 1000, 5000]
for idx, val in enumerate(N):
    exp_path = f"_4_{val}"
    data, N, d, a = read_data(exp_path)
    clf = GMM(data, 5, N, d)
    results = clf.avg_means_over_runs()
    print(f"N {N}")
    print(f"Nominal means {np.unique(a)}")
    print(f"Results for {k} | {results}")
    s_d = calculate_std_dev_euclidean(results, data, k, N,d)
    #std_dev_exp1.append(s_d)
    mean_assignments = get_assignments(results, data, k, simple_distance)
    #print(mean_assignments)
    lop = likelihood_of_pairs(mean_assignments, a)
    print(f"Standard Deviations {s_d}")
    print(f"Mean Standard Deviation across {k} clusters: {np.mean(s_d)}")
    print(f"Likelihood of pairs {lop}")
    #lop_exp_1.append(lop)
    print()
    #avg_std_dev_exp1.append(np.mean(s_d))

Data Set: Experiment_4_100
N 100
Nominal means [1.   2.25 3.5  4.75 6.  ]
Results for 5 | [[0.59139341]
 [2.51984854]
 [3.66748658]
 [4.83133308]
 [6.23952299]]
Standard Deviations [3.2074343722882057, 3.1371998213967345, 2.6413163155794166, 3.1225758695711443, 3.026034676090677]
Mean Standard Deviation across 5 clusters: 3.0269122109852353
Likelihood of pairs 0.4787878787878788

Data Set: Experiment_4_1000
N 1000
Nominal means [1.   2.25 3.5  4.75 6.  ]
Results for 5 | [[0.95018427]
 [2.77064859]
 [3.3185119 ]
 [4.11339384]
 [5.78186937]]
Standard Deviations [10.80060209387416, 9.27737005833878, 7.460386795047738, 9.563277130311604, 12.09070582019269]
Mean Standard Deviation across 5 clusters: 9.838468379552994
Likelihood of pairs 0.5376191989828353

Data Set: Experiment_4_5000
N 5000
Nominal means [1.   2.25 3.5  4.75 6.  ]
Results for 5 | [[1.21390002]
 [2.77797182]
 [3.5340977 ]
 [4.27399269]
 [5.94561974]]
Standard Deviations [25.251666962278694, 21.671040254287742, 17.52023029676